In [201]:
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [202]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatOpenAI(model='gpt-4')

# llm = OpenAI(model='gpt-4-vision-preview')

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory,
)

# chat_llm_chain.predict(human_input="my name is angad")

In [203]:
# from langchain.schema.messages import HumanMessage, SystemMessage

# chat = ChatOpenAI(model="gpt-4-vision-preview")
# chat.invoke(
#     [
#         HumanMessage(
#             content=[
#                 {"type": "text", "text": """I'm giving you an image of a floorplan.
#                  """},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://www.grundriss-schmiede.de/images/buerogrundriss/buerogrundriss.png",
#                         "detail": "auto",
#                     },
#                 },
#             ]
#         )
#     ]
# )

In [204]:
def split_string(input_string, max_length=180):
    # max_length = 125
    chunks = []

    current_chunk = ""
    for char in input_string:
        if char == '\n' or len(current_chunk) == max_length:
            chunks.append(current_chunk)
            current_chunk = ""
        else:
            current_chunk += char

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [205]:
start_x = [31792.864927,
31792.864927,
31792.864927,
24204.196911,
23086.27139,
24447.946444,
24447.946444,
20588.750971,
20588.750971,
19556.033119,
21498.338399,
19178.890458,
19178.890458,
11325.642678,
11325.642678,
15559.078562,
16404.648802,
9701.790319,
15138.40578,
5999.464942,
24477.237227,
24477.237227,
21498.338399,
19178.890458,
19178.890458,
16404.648802,
13996.418764,
15138.40578,
11093.15454,
21437.644176,
19789.016963,
24447.946444,
31792.864927,
34635.32564,
5999.464942,
3222.696981,
23086.27139,
23086.27139,
19789.016963,
19789.016963,
23086.27139,
11093.15454,
11093.15454,
5999.464942,
5999.464942,
9174.174248,
11093.15454,
552.603696,
37394.984283,
37394.984283,
552.603696,
3498.761184,
9391.632409,
9391.632409,
3498.761184]

In [206]:
end_x = [31792.864927,
31792.864927,
31792.864927,
24447.946444,
23626.664256,
24447.946444,
24447.946444,
20588.750971,
20588.750971,
19178.890458,
20133.565773,
19178.890458,
19178.890458,
11325.642678,
11325.642678,
11325.642678,
16136.611217,
9391.632409,
10279.322973,
5999.464942,
24477.237227,
20588.750971,
21498.338399,
19178.890458,
16404.648802,
16404.648802,
13996.418764,
15138.40578,
11093.15454,
21437.644176,
11093.15454,
31792.864927,
34635.32564,
34635.32564,
3222.696981,
3222.696981,
23086.27139,
19789.016963,
19789.016963,
23086.27139,
23086.27139,
11093.15454,
5999.464942,
5999.464942,
8596.641594,
11093.15454,
11093.15454,
37394.984283,
37394.984283,
552.603696,
552.603696,
9391.632409,
9391.632409,
3498.761184,
3498.761184]

In [207]:
start_y = [-2379.722858,
1493.872839,
3835.711592,
15.444393,
15.444393,
2654.279001,
3835.711592,
7404.962615,
5132.58684,
7633.952267,
7633.952267,
6706.490993,
7633.952267,
8399.596468,
7355.592694,
7355.592694,
7355.592694,
5132.58684,
5132.58684,
-2189.956245,
11520.517391,
5132.58684,
11520.517391,
7633.952267,
5132.58684,
5132.58684,
11520.517391,
7355.592694,
-527.87009,
15.444393,
3835.711592,
3835.711592,
-941.851185,
-941.851185,
-527.87009,
-527.87009,
2063.410443,
3835.711592,
3835.711592,
15.444393,
15.444393,
3150.140023,
3835.711592,
3835.711592,
-527.87009,
-527.87009,
-527.87009,
-4561.199065,
-4561.199065,
11520.517391,
11520.517391,
5132.58684,
5132.58684,
7628.666651,
7628.666651]

In [208]:
end_y = [-4561.199065,
-1727.143587,
2247.601897,
15.444393,
15.444393,
-4561.199065,
3408.008059,
7633.952267,
6651.233557,
7633.952267,
7633.952267,
5132.58684,
7460.220051,
11520.517391,
7645.86741,
7355.592694,
7355.592694,
5132.58684,
5132.58684,
-4561.199065,
5132.58684,
5132.58684,
7633.952267,
11520.517391,
5132.58684,
11520.517391,
7355.592694,
5132.58684,
-4561.199065,
-4561.199065,
3835.711592,
3835.711592,
-941.851185,
-4561.199065,
-527.87009,
-4561.199065,
3835.711592,
3835.711592,
15.444393,
15.444393,
1309.681385,
3835.711592,
3835.711592,
-527.87009,
-527.87009,
-527.87009,
2396.410965,
-4561.199065,
11520.517391,
11520.517391,
-4561.199065,
5132.58684,
7628.666651,
7628.666651,
5132.58684]

In [209]:
prompt = """Imagine an x-y cartesian grid with precision of 1 metre. Point A is at (2385,4933) and Point B is at (34406,4933).
What is the distance from point A to point B."""
for i in range(len(start_x)-40):
    s = "Rule {} : YOU CANNOT CROSS THE LINE FROM ({},{}) TO ({},{})".format(i+1,int(start_x[i]),int(start_y[i]),int(end_x[i]),int(end_y[i]))
    prompt = prompt + s + "\n"

prompt = prompt + """AT EACH STEP, GO THROUGH ALL RULES TO CHECK IF EVERY RULE IS BEING OBEYED. EXPLAIN
THE ANSWER FOR EACH RULE. YOU CAN MOVE DIAGONALLY.
YOU CAN ALSO MOVE JUST BESIDE THE WALLS/OBSTACLES."""

In [210]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter


loader = TextLoader("fire_life_safety.md")
md_text = loader.load()[0].page_content



In [212]:
# q_string = """Imagine an x-y cartesian grid with precision of 1 metre. Point A is at (100,100) and Point B is at (500,200).
# What is the distance from point A to point B.
# There are the following 3 rules to be obeyed during the journey.
# Rule 1: YOU CANNOT CROSS THE LINE FROM (200,0) TO (200,200). THIS WALL HAS AN OPENING FROM (200,20) TO (200,30).
# Rule 2: YOU CANNOT CROSS THE LINE FROM (200,200) TO (300,200).
# Rule 3: YOU CANNOT CROSS THE LINE FROM (400,100) TO (400,300).
# Rule 4: 0<x<600 & 0<y<300 for any point during the journey.
# Rule 5: yOU CAN GO TO THE OTHER SIDE OF THE WALLS THROUGH THE OPENINGS IF MENTIONED. CHECK If going through them is the shorter path, do it.

# AT EACH STEP, GO THROUGH ALL RULES TO CHECK IF EVERY RULE IS BEING OBEYED. EXPLAIN
# THE ANSWER FOR EACH RULE. YOU CAN MOVE DIAGONALLY.
# YOU CAN ALSO MOVE JUST BESIDE THE WALLS/OBSTACLES."""

q_string = """
1. Worked as a part of The Pissarides Review into the Future of Work and Wellbeing (Website) led by Nobel Prize-
winning economist Professor Sir Christopher Pissarides. The review aims to quantify the amount of technological disruption in UK using a measure named Disruption Index (DI).
2. Utilised Partial Distance Correlation to consider confounding variables and analyze the relationship among variables used to calculate DI.
3. Performed data analysis to extract valuable insights to identify the technological evolution of different geographic areas
4. Conducted clustering analysis on a graph where nodes represented skills extracted from job advertisements. Utilized the
severability method (arxiv/severability) to discern evolving skill clusters over time within a specific profession
5. Modified the open source code for the severability package (github/severability) to remove bugs and add functionality

Reduce the word count for each point.
"""


q_string = """
1. Compared the new EPIC Deterioration Index mortality prediction model with the existing MEWS model in the hospital to
assess the benefits of adopting the new prediction model.
2. Determined optimum model thresholds for best model performance for various medical departments.
3. Demonstrated that the new model performed better in terms of true positives and false negatives.

Summarise these points in less than 25 words

"""


q_string = """
1. Developed a C# desktop telemetry app to receive and analyze real-time data from a student F1 car.
2. Utilised User Datagram Protocol (UDP) to transfer data from the car over wifi at a rate of 40 Hz.
3. Utilised Matlab and Simulink to set up the transmission of data in UDP packets from the car.

Summarise these points in less than 25 words
"""
# 4. Conducted clustering analysis on a graph where nodes represented skills extracted from job advertisements. 
large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

Sure, how can I assist you further?


In [16]:
q_string = """give the path. even if its an estimate"""
# Provide me with a detailed description or visualization of the resulting maze in the X-Y plane format.
large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)


As a chatbot, I'm not equipped to visually analyze and calculate the exact path on an x-y Cartesian grid based on the numerous rules provided, especially in a text-based environmen
. This task requires a specific type of software developed for graphical or spatial analysis, like pathfinding algorithms in a GIS (Geographic Information System) software, or algo
ithms like A* or Dijkstra's algorithm.

However, I can provide a general strategy:

1. Start at point A (2385,4933).
2. Navigate around the restrictions by moving toward the next available point that is closest to point B (34406,4933) without crossing any restricted lines.
3. Continue this process, always moving toward point B while avoiding restricted lines, until you reach point B.

Please note that the exact path would depend on the specific configuration and placement of the restrictions, and therefore cannot be accurately determined without using a graphica
 representation or spatial analysis software.

It's recommended to us

In [12]:
q_string = """can i give u .bxf or .dwg files"""

large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

I'm sorry for the confusion, but as a text-based AI, I don't have the ability to process files or view content like a human would. I can provide information and answer questions ba
ed on the text inputs I receive. If you have specific questions about your .bxf or .dwg files, feel free to ask and I'll do my best to help!
